In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Reusable classes
import sys
sys.path.append("/home/jovyan/work/upload/ml-experiments/sc_wrangling")

# Data path
import pathlib
FOLDER = pathlib.Path("/home/jovyan/work/upload/")
TABLE_PATH = FOLDER / 'data/pickle/df_columns_labels.pkl'
LAW2VEC_PATH = FOLDER / 'data/law2vec/Law2Vec.200d.txt'

# Manage ML lifecycle
import mlflow
from mlflow import log_params, set_tags, log_metrics
MLFLOW_TRACKING_URI = 'http://srv.meaningfy.ws:8989'

# Math stuff
import numpy as np
from numpy.core.records import ndarray

# Data visualisation
import pandas as pd
from pandas import Series

# Vectorize algotrithm
from gensim.models import Word2Vec, KeyedVectors

# Random Forest Algorithm
from sklearn.ensemble import RandomForestClassifier as RFC
# :criterion: The function to measure the quality of a split
CRITERION = "entropy"
# :min_samples_split:The minimum number of samples
# required to split an internal node
MIN_SAMPLES_SPLIT = 3

# Intermediate steps of the pipeline must be ‘transforms’,
# that is, they must implement fit and transform methods.
from sklearn.pipeline import Pipeline

# Splitting data into train and test
from sklearn.model_selection import train_test_split
RANDOM_STATE = 42
# Where "train" contains 70% of data and "test" - 30%
TEST_SIZE = 0.3
SHUFFLE = True

# Mean embedding function
from mean_vectorizer import MeanEmbeddingVectorizer

# Metrics Evaluation Methods
from evaluation_metrics import model_evaluation_metrics

# Transform DataFrame to dictionary
from dictionary_transformation import series_pair_to_dict

In [2]:
# Read data
df = pd.read_pickle(TABLE_PATH)
df.head()

,Concatenated Data (clean),Category,Subcategory,Type of measure,Target groups,One person or microenterprises|Self-employed|Solo-self-employed,SMEs|Sector specific set of companies,One person or microenterprises|SMEs,Employees in standard employment|Other groups of workers,Employees in standard employment|Larger corporations,...,Children (minors)|Disabled|Older citizens|Parents|SMEs|Single parents|Single parents in employment|The COVID-19 risk group|Workers in care facilities|Workers in essential services,Employees in standard employment|Workers in care facilities|Workers in non-standard forms of employment,Companies providing essential services|Workers in essential services,Contractors of a company,Other businesses|Unemployed,Seasonal workers|Workers in non-standard forms of employment,Employees in standard employment|Particular professions,Businesses,Citizens,Workers
0,"[hardship, case, fund, safety, net, selfemploy...",2,12,0,One person or microenterprises|Self-employed|S...,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
1,"[state, support, tourism, access, financeas, t...",0,0,0,SMEs|Sector specific set of companies,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,"[bank, guarantees, smes, oneperson, enterprise...",0,0,0,One person or microenterprises|SMEs,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,"[emergency, measures, relating, shorttime, wor...",6,14,0,Employees in standard employment|Other groups ...,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1
4,"[airbus, agreement, making, unworked, hours, p...",3,7,2,Employees in standard employment|Larger corpor...,0,0,0,0,1,...,0,0,0,0,0,0,0,1,1,1


In [3]:
# Independent data
columns = df['Concatenated Data (clean)']
# Label data
target_groups_classes = df[['Businesses', 'Citizens', 'Workers']]

law2vec model

In [4]:
# Load a word2vec model stored in the C *text* format.
model = KeyedVectors.load_word2vec_format(LAW2VEC_PATH, binary=False)
l2v_dict = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}

In [31]:
# Use pipes to implement steps of fit and transform method
rfc_l2v = Pipeline([
    # Add the words we want to mean
    ("word2vec vectorizer", MeanEmbeddingVectorizer(l2v_dict)),
    ("Random Forest", RFC(criterion=CRITERION))])

In [6]:
# Split data into test and train sets
X_train, X_test, y_train, y_test = train_test_split(
    columns, target_groups_classes, random_state=RANDOM_STATE, test_size=TEST_SIZE, shuffle=SHUFFLE)

In [32]:
# Fit Random Forest model with out train data
rfc_l2v.fit(X_train, y_train)

Pipeline(steps=[('word2vec vectorizer',
                 <mean_vectorizer.MeanEmbeddingVectorizer object at 0x7f83a1502370>),
                ('Random Forest', RandomForestClassifier(criterion='entropy'))])

In [33]:
# Model Evaluation
rfc_l2v.score(X_train, y_train)

1.0

In [34]:
# Random Forest prediction based on test data
prediction = rfc_l2v.predict(X_test)

In [35]:
# View evaluation metrics table
evaluation = model_evaluation_metrics(y_test, prediction)
evaluation

,Metric Labels,Metric values
0,Accuracy,0.594306
1,Precision,0.774840
2,Recall,0.609479
3,F1-Score,0.665055
4,Mean Absolute Error,0.225386
5,Mean Squared Error,0.225386


In [36]:
# Transform columns data to dict to use in MLFlow
transformation = series_pair_to_dict(evaluation, 'Metric Labels', 'Metric values')
transformation

{'Accuracy': 0.594306049822064,
 'Precision': 0.7748403575989782,
 'Recall': 0.6094789806849538,
 'F1-Score': 0.6650550631211388,
 'Mean Absolute Error': 0.2253855278766311,
 'Mean Squared Error': 0.2253855278766311}

In [37]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="Target groups classes (Random Forest, Law2Vec)")

with mlflow.start_run():

    parameters = {"Language model": 'law2vec',
                  "Random state": RANDOM_STATE,
                  "Test size": TEST_SIZE,
                  "Shuffle": SHUFFLE,
                  "Criterion": CRITERION}
    log_params(parameters)
    log_metrics(transformation)
